In [1]:
import numpy as np
from matplotlib.pyplot import *
from tqdm import tqdm
from astropy.cosmology import Planck13

import sys
sys.path.append('/home/samantha/Documents/REU/scripts/') # Use the absolute path to the directory
%pwd
from prospectFunctionsSFH import *

from prospect.plotting.sfh import parametric_sfr
from prospect.models.templates import TemplateLibrary
from prospect.models.transforms import tburst_from_fage

In [2]:
sys.path

['/home/samantha/anaconda3/envs/reu/lib/python313.zip',
 '/home/samantha/anaconda3/envs/reu/lib/python3.13',
 '/home/samantha/anaconda3/envs/reu/lib/python3.13/lib-dynload',
 '',
 '/home/samantha/anaconda3/envs/reu/lib/python3.13/site-packages',
 '/home/samantha/Documents/REU/scripts/',
 '/home/samantha/anaconda3/envs/reu/lib/python3.13/site-packages/setuptools/_vendor']

In [23]:
keys = ['zred','mass','logzsol','dust2',
        'sfh','imf_type','tau','tage',
        'add_burst','fburst','fage_burst','tburst',
        'const','add_trunc','sf_start','sf_trunc']
values = [False for x in range(len(keys))]
values
all_params = dict(zip(keys, values))

all_params['tau'] = True
all_params['tage'] = True
all_params['fburst'] = True
all_params['fage_burst'] = True
all_params['const'] = True
all_params['sf_start'] = True
all_params['sf_trunc'] = True

In [33]:
param_ranges = {'zred': {'is_free':False, 'init':0, 'prior':[0,0]},
                'mass': {'is_free':False, 'init':0, 'prior':[0,0]},
                'logzsol': {'is_free':False, 'init':0, 'prior':[0,0]},
                'dust2': {'is_free':False, 'init':0, 'prior':[0,0]},
                'sfh': {'is_free':False, 'init':0, 'prior':[0,0]},
                'imf_type': {'is_free':False, 'init':0, 'prior':[0,0]},
                'tau': {'is_free':False, 'init':0, 'prior':[0,0]},
                'tage': {'is_free':False, 'init':0, 'prior':[0,0]},
                'add_burst': {'is_free':False, 'init':0, 'prior':[0,0]},
                'fburst': {'is_free':False, 'init':0, 'prior':[0,0]},
                'fage_burst': {'is_free':False, 'init':0, 'prior':[0,0]},
                'tburst': {'is_free':False, 'init':0, 'prior':[0,0]},
                'const': {'is_free':False, 'init':0, 'prior':[0,0]},
                'add_trunc': {'is_free':False, 'init':0, 'prior':[0,0]},
                'sf_start': {'is_free':False, 'init':0, 'prior':[0,0]},
                'sf_trunc': {'is_free':False, 'init':0, 'prior':[0,0]}
               }

{'is_free': False, 'init': 0, 'prior': [0, 0]}

In [ ]:
def getParams(all_params):
    
    # Fixed parameters
    run_params = {}
    run_params['zred'] = 3.548 # Redshift of the cliff
    run_params['mass'] = 1e8
    run_params['logzsol'] = 0.0 # Solar metallicity
    run_params['dust2'] = 0.0 # No dust in the galaxy
    
    run_params['sfh'] = 1 # tau model plus a constant component and a burst (4 for delay tau, unecessary for our purpose)
    run_params['imf_type'] = 2 # (Kroupa 2001)

    param_priors = {'zred': {'is_free':False, 'prior':[0,0]},
                    'mass': {'is_free':False, 'prior':[0,0]},
                    'logzsol': {'is_free':False, 'prior':[0,0]},
                    'dust2': {'is_free':False, 'prior':[0,0]},
                    'sfh': {'is_free':False, 'prior':[0,0]},
                    'imf_type': {'is_free':False, 'prior':[0,0]},
                    'tau': {'is_free':False, 'prior':[0,0]},
                    'tage': {'is_free':False, 'prior':[0,0]},
                    'add_burst': {'is_free':False, 'prior':[0,0]},
                    'fburst': {'is_free':False, 'prior':[0,0]},
                    'fage_burst': {'is_free':False, 'prior':[0,0]},
                    'tburst': {'is_free':False, 'prior':[0,0]},
                    'const': {'is_free':False, 'prior':[0,0]},
                    'add_trunc': {'is_free':False, 'prior':[0,0]},
                    'sf_start': {'is_free':False, 'prior':[0,0]},
                    'sf_trunc': {'is_free':False, 'prior':[0,0]}
                   }

    N = 2
    taus = [1e-2, 1]
    tages = np.logspace(-1,np.log10(2),N)
    fbursts = [0.001, .01, .6]
    fages = np.linspace(0,.999,N)
    # consts = np.linspace()
    # starts = np.linspace()
    # truncs = np.linspace()
    
    run_params['tau'] = taus[0] # The e-folding time for the SFH, range is 1e-1 to 1e2
    run_params['tage'] = tages[0] # The age of the host (galaxy) in Gyrs
    
    run_params['add_burst'] = True
    run_params['fburst'] = fbursts[0] # The fraction of mass formed in an instantaneous burst of star formation
    run_params['fage_burst'] = fages[0] # The fraction of the host age at which the burst occurred.
    run_params['tburst'] = tburst_from_fage(**run_params) # The age of the universe (age of the host) when the burst occurred
    # run_params['const'] =  0.8 - run_params['fburst']
    
    run_params['add_trunc'] = False
    # run_params['sf_start'] = starts[0] # Start time of the SFH in Gyrs
    # run_params['sf_trunc'] = truncs[0] # Trunctation time of the SFH in Gyrs. 0.0 => no trunctation

    return

In [26]:
all_params

{'zred': False,
 'mass': False,
 'logzsol': False,
 'dust2': False,
 'sfh': False,
 'imf_type': False,
 'tau': True,
 'tage': True,
 'add_burst': False,
 'fburst': True,
 'fage_burst': True,
 'tburst': False,
 'const': True,
 'add_trunc': False,
 'sf_start': True,
 'sf_trunc': True}

In [12]:
# FIX THESE PARAMETERS
run_params = {}
run_params['zred'] = 3.548 # Redshift of the cliff
run_params['mass'] = 1e8
run_params['logzsol'] = 0.0 # Solar metallicity
run_params['dust2'] = 0.0 # No dust in the galaxy

run_params['sfh'] = 1 # tau model plus a constant component and a burst (4 for delay tau, unecessary for our purpose)
run_params['imf_type'] = 2 # (Kroupa 2001)

#######################
# SET THE LIMITS OF THE PRIORS
N = 2
taus = [1e-2, 1]
tages = np.logspace(-1,np.log10(2),N)
fbursts = [0.001, .01, .6]
fages = np.linspace(0,.999,N)
# consts = np.linspace()
# starts = np.linspace()
# truncs = np.linspace()

run_params['tau'] = taus[0] # The e-folding time for the SFH, range is 1e-1 to 1e2
run_params['tage'] = tages[0] # The age of the host (galaxy) in Gyrs

run_params['add_burst'] = True
run_params['fburst'] = fbursts[0] # The fraction of mass formed in an instantaneous burst of star formation
run_params['fage_burst'] = fages[0] # The fraction of the host age at which the burst occurred.
run_params['tburst'] = tburst_from_fage(**run_params) # The age of the universe (age of the host) when the burst occurred
# run_params['const'] =  0.8 - run_params['fburst']

run_params['add_trunc'] = False
# run_params['sf_start'] = starts[0] # Start time of the SFH in Gyrs
# run_params['sf_trunc'] = truncs[0] # Trunctation time of the SFH in Gyrs. 0.0 => no trunctation

obs = getObs()
sps = getSps(**run_params)

wspec = sps.wavelengths

In [7]:
run_params

{'zred': 3.548,
 'mass': 100000000.0,
 'logzsol': 0.0,
 'dust2': 0.0,
 'sfh': 1,
 'imf_type': 2,
 'tau': 0.01,
 'tage': np.float64(0.1),
 'add_burst': True,
 'fburst': 0.001,
 'fage_burst': np.float64(0.0),
 'tburst': np.float64(0.0),
 'add_trunc': False}

In [8]:
getModel(**run_params)

:::::::
<class 'prospect.models.sedmodel.SpecModel'>

Free Parameters: (name: prior) 
-----------
  mass: <class 'prospect.models.priors.LogUniform'>(mini=100000000.0,maxi=1000000000000.0)
  logzsol: <class 'prospect.models.priors.TopHat'>(mini=-2,maxi=0.19)
  dust2: <class 'prospect.models.priors.TopHat'>(mini=0.0,maxi=2.0)
  tage: <class 'prospect.models.priors.TopHat'>(mini=0.001,maxi=13.8)
  tau: <class 'prospect.models.priors.LogUniform'>(mini=0.1,maxi=30)

Fixed Parameters: (name: value [, depends_on]) 
-----------
  zred: [3.548] 
  sfh: [1] 
  imf_type: [2] 
  dust_type: [0] 
  tburst: [0.] <function tburst_from_fage at 0x7f0db4e28ea0>
  fburst: [0.001] 
  fage_burst: [0.] 
  const: [0.] 

In [4]:
def generateSpecs():
    """Input of all the variable names and which are fixed/which are free with all of the priors"""
    return

-----------------------
### Test cells

In [ ]:
N = 4 # Number of plots we're making to test

fbursts = [0.001, .01, .6]
dusts = np.linspace(0,3,4)
print(fbursts)

specs = np.empty((N,len(wspec)))

for f in tqdm(dusts):
    # run_params['fburst'], run_params['const'] = f, .8 - f
    run_params['dust2'] = f
    model = model_function(**run_params)
    spec, phot, mfrac = model.predict(model.theta, obs=obs, sps=sps)
    specs[n] = spec
print('done')